In [9]:
import glob
import sqlite3
from os import listdir
from os.path import isfile, join

import numpy as np
import pandas as pd

In [6]:
BCU_database_folder = '../data/BCU_database/'

Extract the Photo table from the database

In [7]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect(BCU_database_folder + 'bcu_caption.db')
Photo = pd.read_sql_query("SELECT * from Photo", con)

# Verify that result of SQL query is stored in the dataframe
print(Photo.keys())
print('original number of picture :',len(Photo))

Index(['id', 'bcu_id', 'title', 'title_en', 'caption', 'year_from', 'year_to',
       'photographer_name', 'fund_name', 'locality', 'theme', 'filename',
       'title_en_preprocessed', 'caption_preprocessed'],
      dtype='object')
original number of picture : 2216


Get all the images

In [10]:
mypath = BCU_database_folder + '03_resized'
onlyfiles = [f for f in listdir() if isfile(join(mypath, f))]

In [12]:
files = glob.glob(mypath + '/*')

In [13]:
files_324w = list(filter(lambda k: '324w' in k, files))

In [14]:
missing = []
Photo['filename'] = Photo.loc[~Photo.bcu_id.str.contains('BCUF')].bcu_id.apply(lambda x:  (list(filter(lambda k: x.lower() in k.lower(), files_324w)))[0].removeprefix(mypath + '/') if len(list(filter(lambda k: x.lower() in k.lower(), files_324w))) > 0 else missing.append(x))
print(f'Number of pictures with filenames: {Photo.filename.notnull().sum()}')

Number of pictures with filenames: 1908


Processing titles and captions

In [15]:
Photo['title_en_preprocessed'] = Photo["title_en"].apply(lambda x: np.nan if str(x).find('Souvenir of') != -1 else str(x).lower().replace('portrait of ', '').replace('canton of fribourg','').replace('(fribourg)','').replace('[fribourg]','').replace('of fribourg','').replace('fribourg',''))
Photo['caption_preprocessed'] = Photo["caption"].apply(lambda x: np.nan if (str(x).find('a city in') != -1 or str(x).find('view') != -1) else str(x).replace('portrait of ', '').replace('photograph of ', '').replace('black and white photo of ', ''))
print(f'Number of images with processed captions and filenames: {(Photo.caption_preprocessed.notnull() & Photo.filename.notnull()).sum()}')
print(f'Number of images with processed titles and filenames: {(Photo.title_en_preprocessed.notnull() & Photo.filename.notnull()).sum()}')

Number of images with processed captions and filenames: 1506
Number of images with processed titles and filenames: 1881


In [42]:
Photo.to_sql("Photo",con,if_exists='replace',index=False, index_label="id")

2216